In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

def polynomial_filter(x, y, window,step_size, polyorder):
    n = window//2
    x_data = []
    y_data = []
    y_model = []
    dy_model = []
    d2y_model = []
    rmse = []
    coeff_matrix = np.zeros(
        (
            int((polyorder + 1)), len(range(n,int(len(x)),step_size))
        )
    )
    idx = 0
                            
    for i in range(n,int(len(x)),step_size):
        x_range = x[(i-n):(i+n)]
        y_range = y[(i-n):(i+n)]
        coeffs = np.polyfit(x_range, y_range, polyorder)
        ffit = np.poly1d(coeffs)
        fderiv = ffit.deriv()
        fderiv2 = fderiv.deriv()
        ym = np.polyval(ffit, x_range)
        dym = np.polyval(fderiv, x_range)
        d2ym = np.polyval(fderiv2, x_range)
        coeff_matrix[:,idx] = coeffs
        idx = idx + 1
        
        if i == n:
            for ind in range(0,int(n-step_size),step_size):
                x_data.append(x_range[ind])
                y_data.append(y_range[ind])
                y_model.append(ym[ind])
                dy_model.append(dym[ind])
                d2y_model.append(d2ym[ind])
                rmse.append(np.sqrt(np.mean((ym - y_range)**2)))
        x_data.append(x_range[n])
        y_data.append(y_range[n])
        y_model.append(ym[n])
        dy_model.append(dym[n])
        d2y_model.append(d2ym[n])
        rmse.append(np.sqrt(np.mean((ym-y_range)**2)))        
        if int(len(x_range)) - i < n:
            for ind2 in range(n+step_size,int(n-step_size),step_size):
                x_data.append(x_range[ind2])
                y_data.append(y_range[ind2])
                y_model.append(ym[ind2])
                dy_model.append([dym[ind2]])
                d2y_model.append(d2ym[ind2])
                rmse.append(np.sqrt(np.mean((ym-y_range)**2)))  
        elif int(len(x_range)) - i < n - step_size:
            break

    data = pd.DataFrame()    
    data['x_data'] = pd.Series(data = x_data)
    data['y_data'] = pd.Series(data = y_data)
    data['y_model'] = pd.Series(data = y_model)
    data['dydx'] = pd.Series(data = dy_model)
    data['d2ydx2'] = pd.Series(data = d2y_model)
    data['RMSE'] = pd.Series(data = rmse)
    
    rmse_avg = np.sqrt(np.mean((data['RMSE'])**2))
    avg_error = np.mean(data['RMSE'])

    data['avg_error'] = pd.Series(data=avg_error)
    data['rmse_avg'] = pd.Series(data=rmse_avg)
    data['window'] = pd.Series(data=window)
    data['stepsize'] = pd.Series(step_size)
                
    return data, coeff_matrix

In [ ]:
# Cleaning up Kevin's Data

import pandas as pd
import numpy as np

def fixing_time_str(array):
time = np.zeros(len(gitt1['TestTime']))

for i in range(len(gitt1['TestTime'])):
    if '-' not in gitt1['TestTime'][i]:
        hr = float(gitt1['TestTime'][i].split(':')[0]) * 3600
        mins = float(gitt1['TestTime'][i].split(':')[1]) * 60
        sec = float(gitt1['TestTime'][i].split(':')[2])
        time[i] = hr + mins + sec
    if '-' in gitt1['TestTime'][i]:
        #print(gitt1['TestTime'][i].split('-'))
        day = float(gitt1['TestTime'][i].split('-')[0])*86400
        idx2 = gitt1['TestTime'][i].split('-')[1]
        hr = float(idx2.split(':')[0]) * 3600
        mins = float(idx2.split(':')[1]) * 60
        sec = float(idx2.split(':')[2])
        time[i] = day + hr + mins+ sec

In [ ]:
import pandas as pd
import numpy as np

def pulse_separations(df, current):
    d = {}
    workingcurrent = np.argwhere(current != 0)
    startidx = int(workingcurrent[0])
    cycle = 1

    for i in range(len(workingcurrent)-1):
        if workingcurrent[i + 1] - workingcurrent[i] != 1:
            endidx = int(workingcurrent[i +1] - 1)
            d[cycle] = df[startidx:endidx]
            d[cycle] = d[cycle].reset_index(drop = True)

            startidx = int(workingcurrent[i +1])
            cycle = cycle + 1
    
    return d
                       

In [ ]:
import pandas as pd

def transient_pulses(d):
    d_trans = {}
    dkeys = d.keys()

    for key in dkeys:
        mask = d[key]['Current/mA'] != 0
        d_trans[key] = d[key][mask]
        d_trans[key] = d_trans[key].reset_index(drop = True)

        d_trans[key]['sqrt_time_diff'] = np.sqrt(d_trans[key]['Time(s)'] - d_trans[key]['Time(s)'][0])
    
    return d_trans

In [ ]:
def find_linear_fit(x, y, window)
    window = 240
    n = window//2
    coefs = []
    mses = []
    intcs = []
    starts = []
    ends = []

    for i in range(n,int(len(x)-n)):
        x_range = np.array(x[(i-n):(i+n)]).reshape(-1, 1)
        y_range = np.array(y[(i-n):(i+n)]).reshape(-1, 1)

        lin_model = LinearRegression()
        lin_model.fit(x_range, y_range)
        y_pred = lin_model.predict(y_range)
        coef = lin_model.coef_
        intc = lin_model.intercept_
        mse = mean_squared_error(y_range, y_pred)
        startidx = int(i - n)
        endidx = int(i + n)

        starts.append(startidx)
        ends.append(endidx)
        coefs.append(coef.flatten()[0])
        intcs.append(intc.flatten()[0])
        mses.append(mse)

        data = pd.DataFrame()
        data['start_idx'] = pd.Series(data = starts)
        data['end_idx'] = pd.Series(data = ends)
        data['coefs'] = pd.Series(data = coefs)
        data['intcs'] = pd.Series(data = intcs)
        data['mses'] = pd.Series(data = mses)